In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os
import sqlite3


conn = sqlite3.connect('./WAR_RP9_InProcess/trial-db/Harvest.db')
c = conn.cursor()





In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [3]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WAR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WAR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

big_df['Date'] = pd.to_datetime(big_df['Date'])

harvest_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

harvest_df = harvest_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\WAR_RP6
WAR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP7
WAR_RP7_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP8
WAR_RP8_Compiled.xlsm
P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP2
WAR_RP2_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP3
WAR_RP3_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP4
WAR_RP4_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP5
WAR_RP5_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\WAR_RP9
WAR_RP9_Compiled.xlsm


In [4]:
harvest_df.loc[harvest_df['TicketID'] == 'WAR18051XXXX', 'TicketID'] = 'WAR180518XXXX' # Found some mislabeled tickets

In [5]:
# Time for the mill ticket normalization
# Normalize the Weeks column.
# Normalize the source column
# Normalize the Mill column
# Normalize the Product column

In [6]:
len(harvest_df['Mill'].unique().tolist())

85

In [7]:
len(harvest_df['Mill'].str.lower().unique().tolist())

85

In [8]:
mill_list = harvest_df['Mill'].unique().tolist()
mill_dict = {}
mill_dict['mill'] = mill_list
mill_df = pd.DataFrame(mill_dict)
mill_df['mill_uid'] = mill_df.index
mill_mapper = dict(zip(mill_df['mill'], mill_df['mill_uid']))
harvest_df['mill'] = harvest_df['Mill'].map(mill_mapper)
harvest_df.drop(columns=['Mill'], inplace=True)

In [9]:
harvest_df['Week'] = harvest_df['Week'].astype(str)
harvest_df.loc[harvest_df['Week'] == 'Week3b', 'Week'] = "3"
def weeker(week):
    if "Week" in week:
        x = week.split("k")[1]
        x = int(x)
        return x
    else:
        return int(week)
    
harvest_df['Week_trial'] = harvest_df['Week'].apply(lambda x: weeker(x))

week_list = harvest_df['Week_trial'].unique().tolist()
week_dict = {}
week_dict['week'] = week_list
week_df = pd.DataFrame(week_dict)
week_df['week_uid'] = week_df.index
week_mapper = dict(zip(week_df['week'], week_df['week_uid']))
harvest_df['week'] = harvest_df['Week_trial'].map(week_mapper)

In [10]:
product_list = harvest_df['Product'].unique().tolist()
product_dict = {}
product_dict['product'] = product_list
product_df = pd.DataFrame(product_dict)
product_df['product_uid'] = product_df.index
product_mapper = dict(zip(product_df['product'], product_df['product_uid']))
harvest_df['product'] = harvest_df['Product'].map(product_mapper)

In [11]:
harvest_df

,Date,Ticket,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week,mill,Week_trial,week,product
0,2021-08-02,160048,PPW,26.31,0.000,26.310,M-82_Week 1.pdf,NaN,6,WAR201030TH04,Week01,0,1,0,0
1,2021-08-03,160157,PPW,25.79,0.000,25.790,M-82_Week 1.pdf,NaN,6,WAR201030TH04,Week01,0,1,0,0
2,2021-08-03,160120,PPW,25.32,0.000,25.320,M-82_Week 1.pdf,,6,WAR201030TH04,Week01,0,1,0,0
3,2021-08-03,160155,PPW,25.31,0.000,25.310,M-82_Week 1.pdf,,6,WAR201030TH04,Week01,0,1,0,0
4,2021-08-03,160082,PPW,27.17,0.000,27.170,M-82_Week 1.pdf,,6,WAR201030TH04,Week01,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2024-08-16,292262,PPW,27.93,0.000,27.930,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10,47,10,1,0
1570,2024-08-17,292264,PPW,29.21,0.000,29.210,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10,47,10,1,0
1571,2024-08-16,69692,PST,31.24,0.125,31.115,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10,4,10,1,1
1572,2024-08-16,69753,PST,27.99,0.000,27.990,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10,4,10,1,1


In [12]:
# get ticketid from sql and see how that shakes out
ticketid_df = pd.read_sql('SELECT * FROM ticketid',conn)
ticket_id_mapper = dict(zip(ticketid_df['ticketid'], ticketid_df['id']))
harvest_df['ticketid'] = harvest_df['TicketID'].map(ticket_id_mapper)
harvest_df['ticketid'] = harvest_df['ticketid'].astype(int)

In [13]:
harvest_df.drop(columns=['Product', 'TicketID', 'Week'],inplace=True)
harvest_df.rename(columns={'Tons (U.S)': 'total_tons'})

,Date,Ticket,total_tons,Cull Tons,Accepted Tons,Source,Notes,Reporting Period,mill,Week_trial,week,product,ticketid
0,2021-08-02,160048,26.31,0.000,26.310,M-82_Week 1.pdf,NaN,6,0,1,0,0,121
1,2021-08-03,160157,25.79,0.000,25.790,M-82_Week 1.pdf,NaN,6,0,1,0,0,121
2,2021-08-03,160120,25.32,0.000,25.320,M-82_Week 1.pdf,,6,0,1,0,0,121
3,2021-08-03,160155,25.31,0.000,25.310,M-82_Week 1.pdf,,6,0,1,0,0,121
4,2021-08-03,160082,27.17,0.000,27.170,M-82_Week 1.pdf,,6,0,1,0,0,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2024-08-16,292262,27.93,0.000,27.930,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,47,10,1,0,44
1570,2024-08-17,292264,29.21,0.000,29.210,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,47,10,1,0,44
1571,2024-08-16,69692,31.24,0.125,31.115,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,4,10,1,1,44
1572,2024-08-16,69753,27.99,0.000,27.990,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,4,10,1,1,44


In [14]:
harvest_df.columns = [x.lower() for x in harvest_df.columns]
harvest_df.columns = [x.replace(' ', '_') for x in harvest_df.columns]

In [15]:
harvest_df[harvest_df['reporting_period'] == 2]

,date,ticket,tons_(u.s),cull_tons,accepted_tons,source,notes,reporting_period,mill,week_trial,week,product,ticketid
0,2018-04-02,559174,29.13,0.0,29.13,HMWCFI_RP2_Harvest_Grp1.pdf,NaN,2,61,1,0,12,71
1,2018-04-03,559404,28.36,0.0,28.36,HMWCFI_RP2_Harvest_Grp1.pdf,NaN,2,61,1,0,12,71
2,2018-04-06,560824,30.21,0.0,30.21,HMWCFI_RP2_Harvest_Grp1.pdf,NaN,2,61,1,0,12,71
3,2018-04-04,559994,29.31,0.0,29.31,HMWCFI_RP2_Harvest_Grp1.pdf,NaN,2,61,1,0,12,71
4,2018-04-04,559849,31.89,0.0,31.89,HMWCFI_RP2_Harvest_Grp1.pdf,NaN,2,61,1,0,12,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2018-05-14,573499,33.78,0.0,33.78,HMWCFI_RP2_Harvest_Grp9.pdf,NaN,2,61,9,11,12,71
180,2018-05-14,573763,31.02,0.0,31.02,HMWCFI_RP2_Harvest_Grp9.pdf,NaN,2,61,9,11,12,71
181,2018-05-16,574467,27.87,0.0,27.87,HMWCFI_RP2_Harvest_Grp9.pdf,NaN,2,61,9,11,12,71
182,2018-05-15,574144,30.15,0.0,30.15,HMWCFI_RP2_Harvest_Grp10.pdf,NaN,2,61,10,1,12,71


# Where am I?

What I have:
- Harvest Table where [mill, week, product, ticketid] are normalized.
- mill table
- week table
- product table
- ticketid table


I could also normalize the source, but I don't know if that is worth it at the moment.
The only other thing I could think of needing is a link to the mill tickets. Unfortunately these will move around on our server. I have also read that saving pdfs or imgs as blob in the table can really slow down the db - and there will be tens of thousands. I am not sure if that is a lot or not.

So, how do we save the ticket images in a way that is referencable?
1. Could save them in a folder structure that is referenced relative to the db. Just need to make sure that the ticket folder and the db always stay together
   1. Not great if we want to upgrade to post-gresql
2. 